# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [192]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [193]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta:@carlakiim


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret

    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [194]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter:@carlakiim

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [195]:
#Produto escolhido:
produto = 'KKWBeauty'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [196]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [197]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [198]:
#transfomando em listas
dftreino = pd.read_excel('./{0}.xlsx'.format(produto))
dfteste = pd.read_excel('./{0}.xlsx'.format(produto), sheetname = 'Teste')

treino = []
teste = []

for i in range (t):
    treino.append([dftreino['Treinamento'][i], dftreino['Relevância'][i]]) 

for i in range (n-t):
    teste.append ([dfteste['Teste'][i], dftreino['Relevância'][i]])


___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [203]:
#funcao do produtorio
def produtorio(n):
    total = 1
    for i in range(0, len(n)):
        total *= n[i]
    return total

class classificador:
        
    def __init__ (self,treino,teste):
        self.Fsim = 0 #frequencia de tweets relevantes
        self.Fnao = 0 #frequencia de tweets nao relevantes
        self.totalpalavra = [0,0,0] #quantidade total de palavras, palavras relevantes(sim) e palavras irrelevantes(nao)
        self.listaFreq = [] #[palavra, ftotal, fsim, fnao]
        self.treino = treino
        self.contapalavra = 0
        self.teste = teste
        
    def normalize(self):
        for tweet in self.treino:
            tweet[0] = tweet[0].replace(':', '')
            tweet[0] = tweet[0].replace('"','')
            tweet[0] = tweet[0].replace('()','')
            tweet[0] = tweet[0].replace('@','')
            tweet[0] = tweet[0].replace('rt','')
            tweet[0] = tweet[0].replace('  ','')
            tweet[0] = tweet[0].replace('"','')
            
            #tweet[0] = tweet[0].split(" ")
        
        for tweet in self.teste:
            tweet[0] = tweet[0].replace(':','')
            tweet[0] = tweet[0].replace('"','')
            tweet[0] = tweet[0].replace('()','')
            tweet[0] = tweet[0].replace('@','')
            tweet[0] = tweet[0].replace('rt','')
            tweet[0] = tweet[0].replace('  ','')
            tweet[0] = tweet[0].replace('"','')
            
            #tweet[0] = tweet[0].split(" ")
            
    
    def treinando(self):
        for tweet in self.treino:
            if tweet[1] == 'sim':
                self.Fsim += 1
            else:
                self.Fnao += 1
                
            for palavra in tweet[0]:
                onlist = False
                self.totalpalavra[0] += 1
                for FreqPala in self.listaFreq:
                    if palavra == self.listaFreq[0] and tweet[1] == 'sim':
                        onlist = True
                        self.listaFreq[1] += 1
                        self.listaFreq[2] += 1
                        self.totalpalavra[1] += 1
                        
                    elif palavra == self.listaFreq[0] and tweet[1] == 'não':
                        onlist = True
                        self.listaFreq[1] += 1
                        self.listaFreq[3] += 1
                        self.totalpalavra[2] += 1
                        
                if not onlist:
                    self.contapalavra += 1
                    if tweet[1] == 'sim':
                        self.listaFreq.append([palavra, 1, 1, 0])
                        self.totalpalavra[1] += 1
                    else:
                        self.listaFreq.append([palavra, 1, 0, 1])
                        self.totalpalavra[2] += 1
                            
                            
   

    def analise(self,tweet):
        listaph = []
        listaphsim = []
        listaphnao = []
        Psim = self.Fsim/300 #frequencia de um tweet relevante aparecer dentre todos
        Pnao = self.Fnao/300 #frequencia de um tweet  nao relevante aparecer dentre todos
        for palavra in tweet:
            nalista = False
            for FreqPala in self.listaFreq:
                if FreqPala[0] == palavra:
                    nalista = True
                    Ph = (FreqPala[1]+1)/self.contapalavra   # frequencia de cada palavra
                    Phsim = (FreqPala[2]+1)/(self.contapalavra + FreqPala[2]) # frequencia de cada palavra relevante(sim) 
                    Phnao = (FreqPala[3]+1)/(self.contapalavra + FreqPala[3]) # frequencia de cada palavra irrelevante (nao)
                    listaph.append(Ph)
                    listaphsim.append(Phsim)
                    listaphnao.append(Phnao)
                                       
                if not nalista:
                    listaph.append(1/self.contapalavra)
                    listaphsim.append(1/(self.contapalavra + FreqPala[2]))
                    listaphnao.append(1/(self.contapalavra + FreqPala[3])) 
        print(listaph)            
        PH = produtorio(listaph)
        PHSIM = produtorio(listaphsim)
        PHNAO = produtorio(listaphnao)
        
        Bayessim = (PHSIM * Psim)/PH
        Bayesnao = (PHNAO * Pnao)/PH
        
        if (Bayessim > Bayesnao):
            return 1
        else:
            return 0
        
    def testando (self):
        relerele = 0
        relenao = 0
        naorelerele = 0
        naorelenaorele = 0
        totalacerto = 0
        totalerro = 0
        for tweet in teste:
            if self.analise(tweet[0]) == tweet[1]:
                totalacerto += 1
                if tweet[0]:
                    relerele += 1
                else:
                    relenao += 1
            else:
                totalerro += 1
                if tweet[0]:
                    relenao += 1
                else:
                    naorelenaorele += 1
                        
        print('Total de erros: ' + str(totalerro))
        print('Total de acertos: ' + str(totalacerto))
        print()
        print('Positivos falsos:' + str(relenao))
        print('POsitivos Verdadeiros:' + str(relerele))
        print()
        print('Negativos Falsos:' + str(naorelenaorele))
        print('Negativos Verdadeiros:' + str(naorelerele))
        

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [204]:
obj = classificador(treino, teste)
obj.treinando()
obj.testando()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


ZeroDivisionError: float division by zero

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
